In [82]:
import random
import jieba

class ProcessDocument:
    def __init__(self, documents,stopwords_path):
        self.documents = documents
        self.words = {}
        self.stopwords = set()
        self.load_stopwords(stopwords_path)
    def load_stopwords(self, path):
        with open(path, 'r') as fin:
            for line in fin:
                self.stopwords.add(line.strip())
                
    def is_chn(self, words):
        for w in words:
            if not (u'\u4e00' <= w <= u'\u9fff' or \
               u'a' <= w <= u'z' or u'A' <= w <= u'Z'or \
               u'0' <= w <= u'9'):
                return False
        return True
    
    def process(self):
        texts = []
        for document in self.documents:
            text = []
            document = document.split('\t')[-1]
            for word in jieba.cut(document.strip()):
                if not self.is_chn(word): continue
                if word in self.stopwords: continue
                if len(word) < 2: continue
                print(word)
                if word not in self.words:
                    self.words[word] = 0
                self.words[word] += 1
                text.append(word)
            texts.append(text)
            
        self.words = sorted(self.words.items(), key=lambda x:x[1], reverse=True)
        self.words = [word for word,count in self.words]
        return self.words,texts
    
    def get_word2id(self):
        return dict([(word,i) for i,word in enumerate(self.words)])
    def get_id2word(self):
        return dict(enumerate(self.words))
    
documents = []
with open('sougo_texts2.txt') as fin:
    for line in fin:
        documents.append(line)
pd = ProcessDocument(documents, 'stopwords.txt')
words,texts = pd.process()
word2id = pd.get_word2id()
id2word = pd.get_id2word()

特约记者
魏柯嘉
杭州
报道
毛昱衡
MM
红包
悠然
地进
对局室
古力
电话
不接
迟到
15
分钟
弃权
下午
王汝南
院长
大盘
讲棋
毛毛
古力
对局
放到
上午
提醒
先到
毛毛
坐在
下手
位置
毛毛可
答应
古力
年纪
手座
说话
古力
衬衣
而入
脸上
睡意
用盘
外招
毛毛
看着
古力
穿花
衬衣
晃眼
意见
古力
针锋相对
古力
复仇
毛毛
一场
有趣
舌战
新湖
英才
美女
围棋
对抗赛
西子湖畔
正式
拉开序幕
12
恩怨
毛昱衡
古力
率先
对局
落子
半小时
毛毛
面露难色
大肥角
一条
执白
古力
占据
中途
马晓春
下午
下棋
美眉
旁观
棋局
棋败势
很快
毫不留情
古力
鲸吞
中央
黑龙
毛毛
投子
认输
东靠
西靠
输得
壮烈
毛毛
笑吟吟
手状
还好
一盘棋
古力
棋后
偷着乐
12
终于
手下留情
最狠
水平
英才
横扫
美女
一盘
在握
下午
四大
世界冠军
怜香惜玉
面对
如雷贯耳
世界冠军
MM
大气
一口
对局
姿势
不约而同
双手
规矩
放在
两腿
正襟危坐
罗洗河
下午
先开
国少
小将
李赫
开局
半盘
小猪
搅得
反输
10
小猪
可不
下风
没坏
时间
各一
小时
包干制
一向
落子
如飞
小猪
时间
用心
爱怜
对手
一把
马晓春
杨梓
俞斌
曹又尹
对局
相继
两位
英才
胜出
告终
白棋
倒贴
情况
组长
终局
还好
八九
国少
初段
杨梓
评价
重复
半盘
顽强
观战
华以刚
一旁
杨梓上
官子
紧张
高手
差距
全方位
杨梓下
棋后
说话
轻轻
紧张
曹又尹
道场
听过
九段
授课
俞斌
算是
半个
师父
当天
第一次
正式
对抗
老师
发虚
速度慢
紧张
紧张
一盘
当属
常昊
王香如
我下
无理
危险
欺负
MM
常昊
对局
老婆
张璇
女儿
冲着
宝贝女儿
半盘
王香如
下出
漏招
那棋
不好
可比
想象
中强
常昊
夸奖
16
王香如
害羞
信心
很足
越来越
信心
首轮
横扫
美女
授二子
对抗
英才
棋技上
差距
用时
紧张
美女
一大
掣肘
冠军队
托大
放二子
肯定
九段
不肯
首轮
就让
美女
常昊
担心
危险
美女
信心十足
王汝南
院长
当天
拍板
毛毛
召集人
张璇任
临时
教练
晚上
女队
好好
开个
准备会
心理学家
发现
中年
女白领
心理
危机
少见
美国
全国


In [80]:
print(len(word2id))
print(word2id)
#print(pd.stopwords)

1095
{'行政': 0, '办理': 1, '工作': 2, '古力': 3, '业务': 4, '受理': 5, '大厅': 6, '宏碁': 7, '毛毛': 8, '时间': 9, '心理': 10, '关岛': 11, '中心': 12, '市场': 13, '亿美元': 14, '食品药品': 15, '审批': 16, '美女': 17, '紧张': 18, '家庭': 19, '努力': 20, '建设': 21, '海军陆战队': 22, '记者': 23, '免费': 24, '国家': 25, '监管局': 26, '窗口': 27, '对局': 28, '中年': 29, '女白领': 30, '危机': 31, '事业': 32, '调整': 33, '学习': 34, '整编': 35, '基地': 36, '设施': 37, '桃花': 38, '全球': 39, '戴尔公司': 40, '戴尔': 41, '电脑': 42, '服务中心': 43, '工作人员': 44, '办事': 45, '下午': 46, '英才': 47, '正式': 48, '常昊': 49, '社会': 50, '转移': 51, '日美军': 52, '大学': 53, '考大学': 54, '选择': 55, '告诉': 56, '植物园': 57, '股票': 58, '休息区': 59, '未来': 60, 'MM': 61, '面对': 62, '半盘': 63, '小猪': 64, '10': 65, '王香如': 66, '担心': 67, '发现': 68, '美国': 69, '全国': 70, '竞争': 71, '目标': 72, '关系': 73, '冲突': 74, '相关': 75, '日本': 76, '两国': 77, '预算': 78, '提供': 79, '迁移': 80, '冲绳': 81, '翻新': 82, '计划': 83, '学不动': 84, '成绩': 85, '理想': 86, '服务员': 87, '周末': 88, '开放': 89, '预计': 90, '门票': 91, '医疗': 92, '没想到': 93, '裘园': 94, '几个': 95, '微笑': 96, '技术': 97, '1

In [130]:
class LDAModel:
    def __init__(self,num_topic, word2id,id2word,documents):
        self.K = num_topic
        self.alpha = 50/self.K
        self.beta = 0.01
        self.z = []
        self.word2id = word2id
        self.id2word = id2word
        self.documents = documents
        
    def init_topic(self, n_mk, n_m, n_kt, n_k):
        M = len(self.documents)
        V = len(self.word2id)
        for m in range(M):
            self.z.append([])
            for w in self.documents[m]:
                k = random.randint(0,self.K-1)
                t = word2id[w]
                self.z[m].append(k)
                n_mk[m][k] += 1
                n_m[m] += 1
                n_kt[k][t] += 1
                n_k[k] += 1 
                
    def reSampling(self, m, t, n_mk, n_m, n_kt, n_k):
        
        topic_alpha = self.alpha * self.K
        topic_beta = self.beta * len(self.word2id)
        
        p = [0] * self.K
        for k in range(self.K):
            p[k] = (n_kt[k][t] + self.beta)/(n_k[k] + topic_beta) * (n_mk[m][k] + self.alpha)/(n_m[m] + topic_alpha)
            if k > 1:
                p[k] += p[k-1]
        u = random.random() * p[self.K -1]
        
        new_k = 0
        for i in range(self.K):
            if p[i] >= u:
                new_k = i
                break
        return new_k
    
    def calc_theta(self, n_mk, n_m):
        topic_alpha = self.alpha * self.K
        theta = [[0 for i in range(self.K)] for i in range(len(self.documents))]
        for m in range(len(self.documents)):
            for k in range(self.K):
                theta[m][k] = (n_mk[m][k] + self.alpha) / (n_m[m] + topic_alpha)
        return theta
    
    def calc_phi(self, n_kt, n_k):
        topic_beta = self.beta * len(self.word2id)
        phi = [[0 for i in range(len(self.word2id))] for i in range(self.K)]
        for k in range(self.K):
            for t in range(len(self.word2id)):
                phi[k][t] = (n_kt[k][t] + self.beta) / (n_k[k] + topic_beta)
        return phi
    
    def lda(self, n_mk, n_m, n_kt, n_k, times):
        print('run lda')
        M = len(self.documents)
        for epoch in range(times):
            #print('epoch %d' % epoch)
            for m in range(M):
                for i in range(len(self.documents[m])):
                    k = self.z[m][i]
                    word = self.documents[m][i]
                    t = word2id[word]
                    n_mk[m][k] -= 1
                    n_m[m] -= 1
                    n_kt[k][t] -= 1
                    n_k[k] -= 1

                    new_k = self.reSampling(m, t, n_mk, n_m, n_kt, n_k)
                    n_mk[m][new_k] += 1
                    n_m[m] += 1
                    n_kt[new_k][t] += 1
                    n_k[new_k] += 1
                    self.z[m][i] = new_k
                
        theta = self.calc_theta(n_mk, n_m)
        phi = self.calc_phi(n_kt, n_k)
        return theta, phi
    
    def run_lda(self, times):
        n_mk = [[0 for i in range(self.K)] for i in range(len(self.documents))]
        n_m = [0 for i in range(len(self.documents))]
        n_kt = [[0 for i in range(len(words))] for i in range(len(self.documents))]
        n_k = [0 for i in range(self.K)]
        
        self.init_topic(n_mk, n_m, n_kt, n_k)
        theta,phi = self.lda(n_mk, n_m, n_kt, n_k, times)
        return theta,phi
    
    def get_topic_N(self,phi,N):
        topics_dic = {}
        for k in range(len(phi)):
            dic = {}
            for i in range(len(phi[k])):
                dic[i] = phi[k][i]
            word_prob_list = sorted(dic.items(), key = lambda d:d[1], reverse = True)
            topics_dic[k] = word_prob_list
            print('topic %d' % k, end='\t')
            for word_pro in word_prob_list[0:N]:
                print(self.id2word[word_pro[0]],word_pro[1], end=' ')
            print('\n')
        return topics_dic
    
    def get_doc_topic(self, theta, topics_dic):
        #doc_dic = {}
        for m in range(len(theta)):
            dic = {}
            for i in range(len(theta[m])):
                dic[i] = theta[m][i]
            topic_prob_list = sorted(dic.items(), key = lambda d:d[1], reverse=True)
            #doc_dic['doc ' + str(m)] = topic_prob_list
            print('doc %d ' % m, end = '\t')
            for top_prob in topic_prob_list[:1]:
                #print(topics_dic[top_prob[0]])
                for word_pro in topics_dic[top_prob[0]][0:15]:
                    print(self.id2word[word_pro[0]],word_pro[1], end=' ')
                print('')

In [131]:
lda = LDAModel(9, word2id, id2word, texts)
theta, phi = lda.run_lda(300)
top_dic = lda.get_topic_N(phi,50)
lda.get_doc_topic(theta, top_dic)

run lda
topic 0	时间 0.01285817481338791 关岛 0.01285817481338791 亿美元 0.01125290954330203 家庭 0.009647644273216148 努力 0.009647644273216148 海军陆战队 0.009647644273216148 记者 0.009647644273216148 免费 0.009647644273216148 国家 0.009647644273216148 监管局 0.009647644273216148 窗口 0.009647644273216148 对局 0.008042379003130266 女白领 0.008042379003130266 学习 0.008042379003130266 整编 0.008042379003130266 基地 0.008042379003130266 桃花 0.008042379003130266 全球 0.008042379003130266 电脑 0.008042379003130266 英才 0.006437113733044385 正式 0.006437113733044385 社会 0.006437113733044385 转移 0.006437113733044385 日美军 0.006437113733044385 大学 0.006437113733044385 考大学 0.006437113733044385 选择 0.006437113733044385 告诉 0.006437113733044385 MM 0.004831848462958503 面对 0.004831848462958503 半盘 0.004831848462958503 小猪 0.004831848462958503 10 0.004831848462958503 王香如 0.004831848462958503 担心 0.004831848462958503 发现 0.004831848462958503 美国 0.004831848462958503 全国 0.004831848462958503 目标 0.004831848462958503 关系 0.004831848462958503 相关 0.0048318484629

In [133]:
from gensim import corpora, models

In [134]:
dictionary = corpora.Dictionary(texts)

In [135]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [136]:
texts_tf_idf = models.TfidfModel(corpus)[corpus]
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=9,update_every=0,passes=20)
texts_lda = lda[texts_tf_idf]
res_dic = dict(lda.print_topics(num_topics=9, num_words=10))
print(res_dic)
print('-----------')
count = 0
for doc1 in texts_lda:
    #for doc in doc1:
    print('doc %d ' % count, res_dic[doc1[0][0]])
    count +=1
    print(count, '------------------')

{0: '0.001*"行政" + 0.001*"办理" + 0.001*"工作" + 0.001*"戴尔公司" + 0.001*"业务" + 0.001*"审批" + 0.001*"受理" + 0.001*"大厅" + 0.001*"中心" + 0.001*"戴尔"', 1: '0.023*"桃花" + 0.019*"植物园" + 0.014*"周末" + 0.010*"天气" + 0.010*"园内" + 0.010*"玉兰" + 0.010*"游客" + 0.010*"开放" + 0.010*"湖边" + 0.005*"享受"', 2: '0.019*"行政" + 0.018*"办理" + 0.016*"工作" + 0.014*"大厅" + 0.014*"受理" + 0.012*"业务" + 0.011*"心理" + 0.011*"中心" + 0.010*"食品药品" + 0.010*"审批"', 3: '0.038*"宏碁" + 0.034*"市场" + 0.017*"全球" + 0.017*"未来" + 0.013*"液晶电视" + 0.013*"兰奇" + 0.013*"股票" + 0.013*"平板电视" + 0.009*"业务" + 0.009*"电脑"', 4: '0.001*"免费" + 0.001*"办理" + 0.001*"考大学" + 0.001*"学习" + 0.001*"行政" + 0.001*"工作" + 0.001*"选择" + 0.001*"大学" + 0.001*"服务员" + 0.001*"古力"', 5: '0.017*"关岛" + 0.013*"免费" + 0.013*"海军陆战队" + 0.013*"建设" + 0.011*"基地" + 0.011*"整编" + 0.011*"设施" + 0.009*"日美军" + 0.007*"亿美元" + 0.007*"预算"', 6: '0.026*"古力" + 0.019*"毛毛" + 0.014*"美女" + 0.012*"紧张" + 0.012*"对局" + 0.010*"常昊" + 0.010*"英才" + 0.010*"下午" + 0.007*"小猪" + 0.007*"王香如"', 7: '0.020*"学习" + 0.020*"大学" + 0.020*"考大学" + 